In [1]:
import pandas as pd
import numpy as np
from numpy import sign
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import t
from scipy.stats import multivariate_t
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from math import log
from math import sqrt
from math import asin
from math import pi
import scipy.linalg as spla
import scipy.stats as stats
import statistics
import time
import math

## Загрузка и обработка данных

In [2]:
apple = yf.download('AAPL','2020-01-01','2021-01-01')
intel = yf.download('INTC','2020-01-01','2021-01-01')
amd = yf.download('AMD','2020-01-01','2021-01-01')
moderna = yf.download('MRNA','2020-01-01','2021-01-01')
disney = yf.download('DIS','2020-01-01','2021-01-01')
square = yf.download('SQ','2020-01-01','2021-01-01')
boeing = yf.download('BA','2020-01-01','2021-01-01')
microsoft = yf.download('MSFT','2020-01-01','2021-01-01')
alibaba = yf.download('BABA','2020-01-01','2021-01-01')
paypal = yf.download('PYPL','2020-01-01','2021-01-01')
facebook = yf.download('FB','2020-01-01','2021-01-01')
baidu = yf.download('BIDU','2020-01-01','2021-01-01')
nvidia = yf.download('NVDA','2020-01-01','2021-01-01')
netflix = yf.download('NFLX','2020-01-01','2021-01-01')
tesla = yf.download('TSLA','2020-01-01','2021-01-01')
amazon = yf.download('AMZN','2020-01-01','2021-01-01')
alphabet = yf.download('GOOGL','2020-01-01','2021-01-01')
twitter = yf.download('TWTR','2020-01-01','2021-01-01')
bankofamerica = yf.download('BAC','2020-01-01','2021-01-01')
jpmorgan = yf.download('JPM','2020-01-01','2021-01-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
dataframes = [apple, intel, amd, moderna, disney, square, boeing, microsoft, alibaba, paypal, facebook, baidu, nvidia, netflix, tesla, amazon, alphabet, twitter, bankofamerica, jpmorgan]
companynames = ['apple', 'intel', 'amd', 'moderna', 'disney', 'square', 'boeing', 'microsoft', 'alibaba', 'paypal', 'facebook', 'baidu', 'nvidia', 'netflix', 'tesla', 'amazon', 'alphabet', 'twitter', 'bankofamerica', 'jpmorgan']
index = 0

for company in dataframes:
    del company['High']
    del company['Low']
    del company['Adj Close']
    del company['Volume']
    company['Close'] = company['Close'] - company['Open']
    del company['Open']
    company.columns = [companynames[index]]
    index = index + 1

    
dataframe = pd.concat(dataframes, axis = 1)
dataframe

,apple,intel,amd,moderna,disney,square,boeing,microsoft,alibaba,paypal,facebook,baidu,nvidia,netflix,tesla,amazon,alphabet,twitter,bankofamerica,jpmorgan
Date,,,,,,,,,,,,,,,,,,,,
2019-12-31,0.930000,0.230000,0.790001,0.600000,0.960007,1.410000,0.350006,0.929993,0.100006,0.629997,1.250000,0.870003,1.100002,1.570007,2.666000,5.839966,3.599976,0.189999,0.190002,0.889999
2020-01-02,1.027504,0.599998,2.239998,-0.340000,2.910004,0.840000,4.770020,1.839996,3.169998,1.279999,3.029999,8.729996,0.290001,3.709991,1.152000,23.010010,20.270020,-0.010002,0.290001,1.300003
2020-01-03,0.070000,0.289997,0.570000,-0.130001,0.100006,0.410000,2.130005,0.299988,0.649994,-0.729996,1.459991,-0.779999,0.242500,-0.880005,0.501999,10.469971,13.520020,-0.189999,-0.079998,0.839996
2020-01-06,1.502495,0.340000,0.369999,-0.570002,0.110001,1.209999,4.440002,1.949997,1.750000,2.599998,5.900009,3.160004,1.184998,12.709991,2.213997,42.880005,46.180054,0.410000,0.439999,1.669998
2020-01-07,-0.362495,-0.849998,-1.099998,-0.369999,-0.290009,0.019997,3.019989,-1.740005,-0.009995,-1.830002,0.239990,-1.490005,0.432499,-5.720001,1.531998,2.359985,-5.349976,0.740002,-0.080002,-1.399994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.649994,0.470001,0.009995,-2.550003,-0.260010,-5.949997,-2.470001,1.330002,-6.235001,-1.720001,-1.480011,1.270004,-0.434998,-1.150024,18.780029,-21.209961,5.160034,-0.549999,-0.310001,-0.570000
2020-12-28,2.699997,-0.119999,-1.330002,-8.639999,3.110001,-9.040009,-2.100006,0.510010,5.479996,-7.770004,8.260010,3.449997,-1.625000,2.690002,-10.820007,89.959961,29.049927,-0.570000,-0.100000,-0.480003
2020-12-29,-3.180008,2.169998,-1.040001,6.594002,-2.709991,-9.880005,-2.050003,-2.160004,4.500000,-4.970001,-0.170013,-0.899994,0.182495,10.969971,4.989990,12.060059,-29.469971,-0.149998,-0.160000,-1.189995


## Оценка максимального правдоподобия и функция плотности

In [4]:
mean = np.mean(dataframe, axis = 0)
cov = np.cov(dataframe.T)

In [5]:

def mypdf(eps):
    np.random.seed()
    rand = np.random.random()
    
    if rand <= eps:
        result = np.random.multivariate_normal(mean, cov)
    else:
        result = multivariate_t.rvs(loc = mean, shape = cov, df = 3)
        
    return result

In [6]:
class my_pdf(stats.rv_continuous):
    def _pdf(self,x, eps):
        result = eps*multivariate_normal(x, mean=mean, cov=cov) + (1 - eps)*multivariate_t.pdf(x, loc=mean, shape=cov, df=3)
        return result 

## Сетевая модель и истинный отсечённый граф

In [9]:
def fechnercoef(data1, data2, size):
    mean1 = np.mean(data1, axis = 0)
    mean2 = np.mean(data2, axis = 0)
    result = 0
    for i in range(size):
        x = sign(data1[i] - mean1)
        y = sign(data2[i] - mean2)
        result += (x * y)
    result /= size
    result += 1
    result /= 2
    return result

In [10]:
def network_model(name, dataframe):
    network =[]
    for i in range(len(companynames)):
        new_row = []
        for j in range(len(companynames)):
            if name == 'pearson':               
                x = pearsonr(dataframe[companynames[i]],dataframe[companynames[j]])
                coef = x[0]
            elif name == 'kendall':
                x = kendalltau(dataframe[companynames[i]],dataframe[companynames[j]])
                coef = x[0]
            elif name == 'fechner':
                coef = fechnercoef(dataframe[companynames[i]],dataframe[companynames[j]], len(dataframe[companynames[j]]))
            new_row.append(coef)
        network.append(new_row)

    return network

In [11]:
network_pearson = network_model('pearson', dataframe)
network_kendall = network_model('kendall', dataframe)
network_fechner = network_model('fechner', dataframe)

In [12]:
pearson_border01 = []
pearson_border03 = []
pearson_border05 = []
pearson_border07 = []

kendall_border01 = []
kendall_border03 = []
kendall_border05 = []
kendall_border07 = []

fechner_border01 = []
fechner_border03 = []
fechner_border05 = []
fechner_border07 = []

In [14]:
def pearson_true_edges(network):
    
    for i in range(len(companynames)):
        network[i][i] = 0
    
    for i in range(len(companynames)):
        for j in range(i, len(companynames)):
            if network[i][j] >= 0.1:
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)               
                pearson_border01.append(new_pair)
            if network[i][j] >= 0.3:
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)           
                pearson_border03.append(new_pair)
            if network[i][j] >= 0.5:
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)              
                pearson_border05.append(new_pair)

            if network[i][j] >= 0.7:
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)               
                pearson_border07.append(new_pair)



In [15]:
def fechner_true_edges(network):
    
    for i in range(len(companynames)):
        network[i][i] = 0
    
    for i in range(len(companynames)):
        for j in range(i, len(companynames)):
            if network[i][j] >= 1/2 + (1/pi)*asin(0.1):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                fechner_border01.append(new_pair)
            if network[i][j] >= 1/2 + (1/pi)*asin(0.3):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                fechner_border03.append(new_pair)
            if network[i][j] >= 1/2 + (1/pi)*asin(0.5):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                fechner_border05.append(new_pair)
            if network[i][j] >= 1/2 + (1/pi)*asin(0.7):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                fechner_border07.append(new_pair)
                

In [16]:
def kendall_true_edges(network):
    
    for i in range(len(companynames)):
        network[i][i] = 0
    
    for i in range(len(companynames)):
        for j in range(i, len(companynames)):
            if network[i][j] >= (2/pi)*asin(0.1):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                real_kendall_border01.append(new_pair)
            if network[i][j] >= (2/pi)*asin(0.3):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                kendall_border03.append(new_pair)
            if network[i][j] >= (2/pi)*asin(0.5):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                kendall_border05.append(new_pair)
            if network[i][j] >= (2/pi)*asin(0.7):
                new_pair = []
                new_pair.append(i)
                new_pair.append(j)
                kendall_border07.append(new_pair)
                

In [17]:
pearson_true_edges(network_pearson)
fechner_true_edges(network_fechner)
kendall_true_edges(network_kendall)

## Моделирование наблюдений 

In [18]:
def statmodel(size, eps):
    # генерируем наблюдения
    obs = []
    for i in range(size):
        obs.append(list(mypdf(eps)))
    
    # определяем сетевую модель и выборочный отcечённый граф
    obst = np.array(obs)
    obst = obst.transpose()

    return obst

In [20]:
def pearson_tests(n, border, obs, p):
       
    network = []
    p = (1 - p)/190
    quantile = stats.norm.ppf(p)
    for i in range(len(companynames)):
        new_row = []
        for j in range(len(companynames)):
            result = 0
            mean1 = np.mean(obs[i], axis = 0)
            mean2 = np.mean(obs[j], axis = 0)
            x = 0
            y = 0
            z = 0
            for t in range(n):
                x += (obs[i][t] - mean1) * (obs[j][t] - mean2)
                y += (obs[i][t] - mean1) * (obs[i][t] - mean1)
                z += (obs[j][t] - mean2) * (obs[j][t] - mean2)
            result = x/sqrt(y * z)
            x = log((1+result)/(1-result)) - log((1+border)/(1-border))
            x /= (2*sqrt(n-3))
            if x >= quantile:
                new_row.append(0)
            else:
                new_row.append(1)
        network.append(new_row)
        
        
    return network

#network_exp1


In [21]:
def fechner_tests(n, border, p, obs):
    network = []
    q = 1/2 + (1/pi * asin(border))
    p = (1 - p)/190
    quantile = stats.binom.ppf(p,n,q)
    for i in range(len(companynames)):
        new_row = []
        for j in range(len(companynames)):
            mean1 = np.mean(obs[i], axis = 0)
            mean2 = np.mean(obs[j], axis = 0)
            result = 0
            for z in range(n):
                x = (obs[i][z] - mean1) * (obs[j][z] - mean2)
                if x > 0:
                    result += 1
                else:
                    pass
            new_row.append(result)
        network.append(new_row)
    
    for i in range(len(companynames)):
        for j in range(len(companynames)):
            if network[i][j] >= quantile:
                network[i][j] = 0
            else:
                network[i][j] = 1

    return network

In [22]:
def generate_pearson(n, border, eps, p):
    obst = statmodel(n, eps)

    test_results = pearson_tests(n, border, obst, p)
    counter = 0
    result = 0
    numb_of_edges = 0

    for i in range(len(companynames)):
        for j in range(i, len(companynames)):
            if i != j and test_results[i][j] == 0:
                numb_of_edges += 1
    
    if border == 0.7:
        for i in true_pearson_border07:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_pearson_border07)
        z = len(true_pearson_border07) - counter
        z /= counter
    
    if border == 0.5:
        for i in true_pearson_border05:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_pearson_border05)
        z = len(true_pearson_border05) - counter
        z /= counter
    
    if border == 0.3:
        for i in true_pearson_border03:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_pearson_border03)
        z = len(true_pearson_border03) - counter
        z /= counter
        
    if border == 0.1:
        for i in true_pearson_border01:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_pearson_border01)
        z = len(true_pearson_border01) - counter
        z /= counter
        
    if result == 1:
        pass
    else:
        result = 0
        
    
    return numb_of_edges, result, counter, z


In [23]:
def generate_fechner(n, border, eps, p):
    obst = statmodel(n, eps)
    
    test_results = fechner_tests(n, border, p, obst)
    counter = 0
    result = 0
    numb_of_edges = 0
    
    for i in range(len(companynames)):
        for j in range(i, len(companynames)):
            if i != j and test_results[i][j] == 0:
                numb_of_edges += 1
    
    if border == 0.7:
        for i in true_fechner_border07:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_fechner_border07)
        z = len(true_fechner_border07) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
    
    if border == 0.5:
        for i in true_fechner_border05:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_fechner_border05)
        z = len(true_fechner_border05) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
    
    if border == 0.3:
        for i in true_fechner_border03:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_fechner_border03)
        z = len(true_fechner_border03) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
        
    if border == 0.1:
        for i in true_fechner_border01:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_fechner_border01)
        z = len(true_fechner_border01) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
            
    if result == 1:
        pass
    else:
        result = 0
        
    
    return numb_of_edges, result, counter, z

In [24]:
def kendall_tests(n, border, obs, p):
    
    network = []
    for i in range(len(companynames)):
        new_row = []
        for j in range(len(companynames)):
            new_row.append(3)
        network.append(new_row)
    q = 2/pi * asin(border)
    p = (1 - p)/190
    quantile = stats.norm.ppf(p)
    coef = sqrt((9*n*(n - 1))/(2*(2*n + 5)))
    for i in range(len(companynames)):
        for j in range(i, len(companynames)):
            result = 0
            for s in range(n):
                for t in range(n):
                    result += sign((obs[i][t] - obs[i][s])*(obs[j][t] - obs[j][s]))
            result /= (n*(n-1))
            result -= q
            result *= coef
            if result >= quantile:
                network[i][j] = 0
                network[j][i] = 0
            else:
                network[i][j] = 1
                network[j][i] = 1
        
        
    return network

In [25]:
def generate_kendall(n, border, eps, p):
    obst = statmodel(n, eps)

    test_results = kendall_tests(n, border, obst, p)
    counter = 0
    result = 0
    numb_of_edges = 0
    
    for i in range(len(companynames)):
        for j in range(i, len(companynames)):
            if i != j and test_results[i][j] == 0:
                numb_of_edges += 1
    
    if border == 0.7:
        for i in true_kendall_border07:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_kendall_border07)
        z = len(true_kendall_border07) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
    
    if border == 0.5:
        for i in true_kendall_border05:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_kendall_border05)
        z = len(true_kendall_border05) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
    
    if border == 0.3:
        for i in true_kendall_border03:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_kendall_border03)
        z = len(true_kendall_border03) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
        
    if border == 0.1:
        for i in true_kendall_border01:
            if test_results[i[0]][i[1]] == 0:
                counter += 1
        result = counter/len(true_kendall_border01)
        z = len(true_kendall_border01) - counter
        if counter != 0:
            z /= counter
        else:
            z = -1
            
    if result == 1:
        pass
    else:
        result = 0
    
    return numb_of_edges, result, counter, z


In [45]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(11):
        result = 0
        for j in range(repeats):
            x = generate_pearson(100, border, i/10, 0.9)
            result += x[1]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5],
                                       new_row[6], new_row[7], new_row[8], new_row[9], new_row[10]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1
0,0.1,0.43,0.52,0.58,0.60,0.70,0.73,0.81,0.83,0.87,0.94,1.0
1,0.3,0.46,0.57,0.63,0.66,0.77,0.76,0.85,0.88,0.88,0.97,1.0
2,0.5,0.81,0.87,0.85,0.90,0.88,0.91,0.93,0.96,0.99,1.00,1.0
3,0.7,0.90,0.93,0.96,0.98,0.95,0.97,0.93,0.99,0.98,1.00,1.0


In [46]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(11):
        result = 0
        for j in range(repeats):
            x = generate_pearson(500, border, i/10, 0.9)
            result += x[1]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5],
                                       new_row[6], new_row[7], new_row[8], new_row[9], new_row[10]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1
0,0.1,0.33,0.36,0.41,0.46,0.55,0.58,0.67,0.72,0.81,0.90,0.99
1,0.3,0.35,0.39,0.44,0.47,0.53,0.59,0.66,0.74,0.83,0.91,1.00
2,0.5,0.70,0.76,0.73,0.79,0.80,0.83,0.87,0.91,0.92,0.97,1.00
3,0.7,0.95,0.96,0.98,0.97,0.94,0.97,0.96,0.98,1.00,1.00,1.00


In [47]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(11):
        result = 0
        for j in range(repeats):
            x = generate_pearson(1000, border, i/10, 0.9)
            result += x[1]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5],
                                       new_row[6], new_row[7], new_row[8], new_row[9], new_row[10]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1
0,0.1,0.31,0.37,0.41,0.45,0.49,0.54,0.61,0.73,0.85,0.91,1.0
1,0.3,0.33,0.38,0.42,0.49,0.56,0.60,0.61,0.75,0.86,0.93,1.0
2,0.5,0.74,0.73,0.75,0.77,0.82,0.85,0.89,0.94,1.00,0.97,1.0
3,0.7,0.95,0.93,0.99,0.97,0.94,0.96,0.99,0.99,1.00,1.00,1.0


In [38]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(11):
        result = 0
        for j in range(repeats):
            x = generate_fechner(100, border, i/10, 0.9)
            result += x[1]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5],
                                       new_row[6], new_row[7], new_row[8], new_row[9], new_row[10]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1
0,0.1,1.000000,1.0,1.0,1.000000,0.998851,1.0,1.000000,1.000000,1.0,1.0,1.000000
1,0.3,0.996522,1.0,1.0,0.998261,0.998261,1.0,0.998261,0.998261,1.0,1.0,0.998261
2,0.5,0.995918,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.000000
3,0.7,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.000000


In [32]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(11):
        result = 0
        for j in range(repeats):
            x = generate_fechner(500, border, i/10, 0.9)
            result += x[1]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5],
                                       new_row[6], new_row[7], new_row[8], new_row[9], new_row[10]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1
0,0.1,1.000000,1.000000,0.998851,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.3,0.986087,0.982609,0.982609,0.987826,0.987826,0.989565,0.987826,0.986087,0.989565,0.989565,0.986087
2,0.5,0.995918,1.000000,0.995918,1.000000,0.991837,0.987755,1.000000,0.991837,0.995918,1.000000,0.987755
3,0.7,1.000000,0.971429,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(11):
        result = 0
        for j in range(repeats):
            x = generate_fechner(1000, border, i/10, 0.9)
            result += x[1]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5],
                                       new_row[6], new_row[7], new_row[8], new_row[9], new_row[10]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1
0,0.1,0.998851,1.000000,0.996552,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.998851
1,0.3,0.951304,0.954783,0.963478,0.972174,0.960000,0.960000,0.960000,0.963478,0.963478,0.958261,0.966957
2,0.5,0.987755,0.983673,1.000000,0.995918,0.983673,0.991837,0.991837,0.983673,0.979592,0.979592,0.987755
3,0.7,0.971429,0.971429,1.000000,1.000000,1.000000,1.000000,0.971429,0.971429,1.000000,1.000000,1.000000


In [50]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(11):
        result = 0
        for j in range(repeats):
            x = generate_kendall(100, border, i/10, 0.9)
            result += x[1]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5],
                                       new_row[6], new_row[7], new_row[8], new_row[9], new_row[10]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1
0,0.1,0.98,0.97,0.98,0.99,0.99,0.99,1.0,1.0,0.98,0.98,1.0
1,0.1,0.99,1.00,0.99,0.99,0.99,0.99,1.0,1.0,0.98,0.99,1.0
2,0.1,1.00,1.00,0.99,1.00,1.00,0.99,1.0,1.0,1.00,0.99,1.0
3,0.7,0.99,1.00,0.99,1.00,1.00,1.00,1.0,1.0,1.00,1.00,1.0


In [51]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.2', '0.4', '0.6', '0.8', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(0, 11, 2):
        result = 0
        for j in range(repeats):
            x = generate_pearson(100, border, i/10, 0.9)
            result += x[0]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.2,0.4,0.6,0.8,1
0,0.1,179.3,180.2,180.7,183.4,185.1,189.1
1,0.3,161.8,162.4,167.1,172.6,178.3,180.4
2,0.5,115.3,119.2,119.7,120.3,121.4,122.9
3,0.7,32.1,30.9,29.1,27.4,25.8,23.3


In [33]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.2', '0.4', '0.6', '0.8', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(0, 11, 2):
        result = 0
        for j in range(repeats):
            x = generate_fechner(100, border, i/10, 0.9)
            result += x[0]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.2,0.4,0.6,0.8,1
0,0.1,190.0,189.8,190.0,189.8,189.8,190.0
1,0.3,189.4,189.2,188.4,188.2,188.8,188.4
2,0.5,165.4,174.8,173.0,167.0,166.8,168.0
3,0.7,100.6,96.8,109.8,94.8,106.4,98.8


In [57]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.2', '0.4', '0.6', '0.8', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
for border in borders:    
    for i in range(0, 11, 2):
        result = 0
        for j in range(repeats):
            x = generate_kendall(100, border, i/10, 0.9)
            result += x[0]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.2,0.4,0.6,0.8,1
0,0.1,189.7,189.8,189.7,189.9,189.9,190.0
1,0.3,188.4,188.3,188.7,188.2,188.8,188.9
2,0.5,166.8,170.3,168.7,167.0,166.8,166.0
3,0.7,98.9,97.8,99.3,97.4,98.1,98.8


In [58]:
modeling_results = pd.DataFrame(columns=['y/n', '100', '500', '1000', 'Number of true edges'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
numbers = [100, 500, 1000]
new_row = []
for border in borders:    
    for i in numbers:
        result = 0
        for j in range(repeats):
            x = generate_fechner(i, border, 1, 0.9)
            result += x[0]
        result /= repeats
        new_row.append(result)
    if border == 0.1:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border01)]
    elif border == 0.3:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border03)]
    elif border == 0.5:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border05)]
    elif border == 0.7:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border07)]
    row_index += 1
    new_row.clear()

modeling_results

,y/n,100,500,1000,Number of true edges
0,0.1,190.0,190.0,189.4,162.0
1,0.3,188.8,165.0,149.0,95.0
2,0.5,167.4,102.4,76.4,38.0
3,0.7,88.8,29.8,20.4,5.0


In [59]:
modeling_results = pd.DataFrame(columns=['y/n', '100', '500', '1000', 'Number of true edges'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
numbers = [100, 500, 1000]
new_row = []
for border in borders:    
    for i in numbers:
        result = 0
        for j in range(repeats):
            x = generate_fechner(i, border, 0, 0.9)
            result += x[0]
        result /= repeats
        new_row.append(result)
    if border == 0.1:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border01)]
    elif border == 0.3:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border03)]
    elif border == 0.5:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border05)]
    elif border == 0.7:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border07)]
    row_index += 1
    new_row.clear()

modeling_results

,y/n,100,500,1000,Number of true edges
0,0.1,189.8,189.6,188.4,162.0
1,0.3,187.8,172.8,148.2,95.0
2,0.5,169.6,96.8,74.2,38.0
3,0.7,94.8,29.0,18.8,5.0


In [62]:
modeling_results = pd.DataFrame(columns=['y/n', '100', '500', '1000', 'Number of true edges'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
numbers = [100, 500, 1000]
new_row = []
for border in borders:    
    for i in numbers:
        result = 0
        for j in range(repeats):
            x = generate_pearson(i, border, 1, 0.9)
            result += x[0]
        result /= repeats
        new_row.append(result)
    if border == 0.1:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border01)]
    elif border == 0.3:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border03)]
    elif border == 0.5:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border05)]
    elif border == 0.7:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border07)]
    row_index += 1
    new_row.clear()

modeling_results

,y/n,100,500,1000,Number of true edges
0,0.1,189.4,188.8,187.9,162.0
1,0.3,180.1,147.9,135.7,95.0
2,0.5,122.3,42.5,31.1,38.0
3,0.7,23.7,8.6,7.4,5.0


In [63]:
modeling_results = pd.DataFrame(columns=['y/n', '100', '500', '1000', 'Number of true edges'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
numbers = [100, 500, 1000]
new_row = []
for border in borders:    
    for i in numbers:
        result = 0
        for j in range(repeats):
            x = generate_pearson(i, border, 0, 0.9)
            result += x[0]
        result /= repeats
        new_row.append(result)
    if border == 0.1:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border01)]
    elif border == 0.3:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border03)]
    elif border == 0.5:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border05)]
    elif border == 0.7:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border07)]
    row_index += 1
    new_row.clear()

modeling_results

,y/n,100,500,1000,Number of true edges
0,0.1,181.1,179.6,179.1,162.0
1,0.3,164.9,138.3,127.4,95.0
2,0.5,115.7,49.8,36.1,38.0
3,0.7,32.3,11.2,8.4,5.0


In [64]:
modeling_results = pd.DataFrame(columns=['y/n', '100', '500', '1000', 'Number of true edges'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
numbers = [100, 500, 1000]
new_row = []
for border in borders:
    for i in numbers:
        result = 0
        for j in range(repeats):
            x = generate_kendall(i, border, 1, 0.9)
            result += x[0]
        print(border, i)
        result /= repeats
        new_row.append(result)
    if border == 0.1:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border01)]
    elif border == 0.3:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border03)]
    elif border == 0.5:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border05)]
    elif border == 0.7:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border07)]
    row_index += 1
    new_row.clear()

modeling_results

,y/n,100,500,1000,Number of true edges
0,0.1,189.6,189.3,188.7,162.0
1,0.3,187.2,173.3,147.9,95.0
2,0.5,170.6,93.6,72.9,38.0
3,0.7,91.4,28.0,16.8,5.0


In [65]:
modeling_results = pd.DataFrame(columns=['y/n', '100', '500', '1000', 'Number of true edges'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
numbers = [100, 500, 1000]
new_row = []
for border in borders:
    for i in numbers:
        result = 0
        for j in range(repeats):
            x = generate_kendall(i, border, 0, 0.9)
            result += x[0]
        print(border, i)
        result /= repeats
        new_row.append(result)
    if border == 0.1:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border01)]
    elif border == 0.3:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border03)]
    elif border == 0.5:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border05)]
    elif border == 0.7:
        modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], len(pearson_border07)]
    row_index += 1
    new_row.clear()

modeling_results

,y/n,100,500,1000,Number of true edges
0,0.1,189.8,189.6,188.9,162.0
1,0.3,186.1,175.4,144.6,95.0
2,0.5,173.3,91.6,73.1,38.0
3,0.7,92.9,24.7,16.3,5.0


In [66]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.2', '0.4', '0.6', '0.8', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
result = 0
counter = 0
for border in borders:    
    for i in range(0, 11, 2):
        result = 0
        for j in range(repeats):
            x = generate_pearson(100, border, i/10, 0.9)
            result += x[2]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.2,0.4,0.6,0.8,1
0,0.1,157.3,158.1,159.7,160.8,161.6,162.0
1,0.3,92.3,92.8,93.7,94.2,94.7,95.0
2,0.5,37.3,37.5,37.7,37.8,38.0,38.0
3,0.7,4.9,4.9,4.9,4.9,5.0,5.0


In [68]:
modeling_results = pd.DataFrame(columns=['y/e', '0', '0.2', '0.4', '0.6', '0.8', '1'])
row_index = 0
repeats = 50
borders = [0.1, 0.3, 0.5, 0.7]
new_row = []
result = 0
counter = 0
for border in borders:    
    for i in range(0, 11, 2):
        result = 0
        for j in range(repeats):
            x = generate_pearson(100, border, i/10, 0.9)
            result += x[3]
        result /= repeats
        new_row.append(result)
    modeling_results.loc[row_index] = [border, new_row[0], new_row[1], new_row[2], new_row[3], new_row[4], new_row[5]]
    row_index += 1
    new_row.clear()

modeling_results

,y/e,0,0.2,0.4,0.6,0.8,1
0,0.1,0.03,0.02,0.02,0.01,0.00,0.0
1,0.3,0.03,0.02,0.02,0.01,0.01,0.0
2,0.5,0.02,0.01,0.01,0.01,0.00,0.0
3,0.7,0.01,0.01,0.01,0.01,0.00,0.0
